In [1]:
import os
import sys

sys.path.append("..")
os.chdir("/home/sean/HAI/main_server/sumica")

In [2]:
import time
from utils import db
import datetime

start_time = time.mktime(datetime.date(2018, 1, 16).timetuple())
end_time = time.mktime(datetime.date(2018, 1, 19).timetuple())
results = db.labels.find({"username": "sean", "time": {"$gt": start_time, "$lt": end_time}})
label_types = results.distinct("label")
results = list(results)
print(start_time, end_time)

1516028400.0 1516287600.0


In [3]:
results, label_types

([{'_id': ObjectId('5a5f6b4f1d41c80dbfcf9623'),
   'label': 'test',
   'time': 1516202831.8121428,
   'username': 'sean'},
  {'_id': ObjectId('5a5f720a1d41c87c68eb9146'),
   'label': 'test',
   'time': 1516204554.6357632,
   'username': 'sean'}],
 ['test'])

In [4]:
labels = [r["label"] for r in results]
labels = [(r["time"], label_types.index(r["label"])) for r in results]
labels = {"activity": labels}
labels

{'activity': [(1516202831.8121428, 0), (1516204554.6357632, 0)]}

In [5]:
%load_ext autoreload
%autoreload 2

from flask import Flask

username = "sean"

app = Flask(__name__)
app.config.from_object('config.Config')

with app.app_context():
    from controllers.activitylearner import ActivityLearner

In [6]:
act = ActivityLearner(username, start_thread=False)

In [7]:
act.update_learner()

2018-01-19 04:55:59 ubuntu controllers.activitylearner[10336] DEBUG labels: {'activity': [(1516202831.8121428, 0), (1516204554.6357632, 0)]}
2018-01-19 04:56:00 ubuntu controllers.tests.learner2[10336] DEBUG dataset size: 2491
2018-01-19 04:56:00 ubuntu controllers.tests.learner2[10336] DEBUG already recorded: []
2018-01-19 04:56:00 ubuntu controllers.tests.learner2[10336] DEBUG last fixed: 0, start fit: 0
2018-01-19 04:56:04 ubuntu controllers.tests.learner2[10336] DEBUG new segments: [(0, 6), (7, 10), (11, 19), (20, 27), (27, 41), (41, 46), (46, 63), (63, 71), (71, 76), (76, 87), (87, 96), (96, 106), (107, 109), (110, 111), (112, 112), (113, 116), (117, 118), (119, 119), (120, 120), (121, 124), (124, 131), (131, 134), (135, 144), (144, 153), (153, 161), (161, 168), (168, 172), (172, 174), (174, 189), (189, 194), (195, 213), (213, 230), (230, 240), (240, 249), (249, 255), (255, 261), (262, 264), (264, 266), (266, 275), (275, 279), (280, 282), (283, 320), (320, 347), (347, 364), (364, 

In [9]:
act.update_learner()

2018-01-19 04:57:34 ubuntu controllers.activitylearner[10336] DEBUG labels: {'activity': [(1516202831.8121428, 0), (1516204554.6357632, 0)]}
2018-01-19 04:57:35 ubuntu controllers.tests.learner2[10336] DEBUG dataset size: 2491
2018-01-19 04:57:35 ubuntu controllers.tests.learner2[10336] DEBUG last recorded segment: {'_id': ObjectId('5a60fbd41d41c828602d6d2d'), 'username': 'sean', 'start_time': 1516206219.5931327, 'end_time': 1516206226.847426}
2018-01-19 04:57:35 ubuntu controllers.tests.learner2[10336] DEBUG already recorded: [[0, 6], [7, 10], [11, 19], [20, 27], [27, 41], [41, 46], [46, 63], [63, 71], [71, 76], [76, 87], [87, 96], [96, 106], [107, 109], [110, 111], [112, 112], [113, 116], [117, 118], [119, 119], [120, 120], [121, 124], [124, 131], [131, 134], [135, 144], [144, 153], [153, 161], [161, 168], [168, 172], [172, 174], [174, 189], [189, 194], [195, 213], [213, 230], [230, 240], [240, 249], [249, 255], [255, 261], [262, 264], [264, 266], [266, 275], [275, 279], [280, 282], 